In [91]:
import pandas as pd
from tqdm import tqdm

In [92]:
df_sents = pd.read_parquet("sentences.parquet")

In [95]:
big_word_dict = {}
for i, row in tqdm(df_sents.iterrows()):
    words = row["sentence_obj"]
    for word in words:
        key = f"{word['string']}_{word['pos']}_{word['features']}_{word['category']}"
        if key not in big_word_dict:
            big_word_dict[key] = {
                "text": word["string"],
                "count": 1,
                "lemma": word["lemma"],
                "pos": word["pos"],
                "features": word["features"],
                # "category": word["category"]
            }
        else:
            big_word_dict[key]["count"] += 1

134645it [00:09, 13662.24it/s]


In [96]:
word_to_lemma_df = pd.DataFrame.from_dict(big_word_dict, orient="index").sort_values(by="count", ascending=False).reset_index()#.drop([",", ".", ";", "·"])
word_to_lemma_df = pd.concat([word_to_lemma_df.drop(['features'], axis=1), word_to_lemma_df['features'].apply(pd.Series)], axis=1)
word_to_lemma_df = word_to_lemma_df[word_to_lemma_df["count"] > 1]
# word_to_lemma_df = pd.concat([word_to_lemma_df.drop(['category'], axis=1), word_to_lemma_df['category'].apply(pd.Series)], axis=1)

In [101]:
word_to_lemma_df = word_to_lemma_df[word_to_lemma_df.text.str.isalpha()]

In [103]:
word_to_lemma_df.to_parquet("word_to_lemma.parquet")